In [ ]:
import torch
import pandas as pd
import numpy as np
import random

import os

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00


In [ ]:
!wget https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/Data/train.csv
!wget https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/Data/eval.csv

--2023-10-13 12:09:52--  https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/Data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168980 (165K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 165.02K  --.-KB/s    in 0.03s   

2023-10-13 12:09:53 (4.91 MB/s) - ‘train.csv’ saved [168980/168980]

--2023-10-13 12:09:53--  https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/Data/eval.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [ ]:
df = pd.read_csv('./train.csv')

In [ ]:
df.sample(10)

,item_id,shop_id,comment,rate_star,label
1410,22554314017,76448301,"Shop tư_vấn nhiệt_tình , cb chat với shop để đ...",5,0
28,21892301958,76448301,"Không đẹp , logo in lệch , tai nghe to làm đau...",1,1
86,20542845170,76448301,"Sạc lâu , lúc đầu tưởng do dây sạc có vấn_đề n...",1,1
578,18344910325,76448301,Chất_lượng sản_phẩm : tay nghe Bluetooth .,5,0
887,20784041843,76448301,"Tai nghe chính hãng , kết_nối nhanh , âm_thanh...",5,0
1123,22154458199,76448301,"Hàng nguyên seal , chính hãng VNA , nhưng cảm_...",5,0
1039,23080283521,76448301,Nay mình sạc thử lần đầu thấy đt lên khá nhanh .,5,0
1297,15788053880,76448301,Tính_năng nổi_bật : nghe eme nghe dc 2 tieng .,5,0
362,22881480182,76448301,Sản_phẩm âm_thanh nge khá ổn .,4,0
626,16223230656,76448301,Chất_lượng sản_phẩm : Loa to .,5,0


In [ ]:
sentences = df['comment'].values
labels = df['label'].values

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", do_lower_case=True)

In [ ]:
print(sentences[296])
print(tokenizer.tokenize(sentences[296]))
print(tokenizer(sentences[296]))

Chất_lượng sản_phẩm : tạm đc .
['Chất_lượng', 'sản_phẩm', ':', 'tạm', 'đc', '.']
{'input_ids': [0, 6869, 265, 27, 918, 51744, 5, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
max_len = 0

for sentence in sentences:
  input_ids = tokenizer.encode(sentence)
  max_len = max(max_len, len(input_ids))

print(max_len)

89


In [ ]:
input_ids = []
attention_masks = []

for sent in sentences:
  encoded_dict = tokenizer.encode_plus(
      sent,
      max_length=100,
      padding='max_length',
      return_attention_mask=True,
      truncation=True
  )

  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])

In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

labels = torch.tensor(labels)

In [ ]:
print(sentences[0])
print(input_ids[0])
print(attention_masks[0])
print(labels[0])

Chất_lượng sản_phẩm : giao sai sản_phẩm .
tensor([   0, 6869,  265,   27,  574, 1069,  265,    5,    2,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
from torch.utils.data import TensorDataset, random_split, DataLoader

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.9 * len(dataset))
val_size = int(len(dataset) - train_size)

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(train_size, val_size)

1358 151


In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW

model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base-v2",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

model.cuda()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
total_steps = len(train_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

In [ ]:
def flat_acc(pred, labels):
  pred_flat = np.argmax(pred, axis=1).flatten()
  labels_flat = labels.flatten()

  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def all_seed(seed_val):
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

In [ ]:
all_seed(42)

for epoch in range(0, epochs):
  print('Epoch {} in {} total'.format(epoch + 1, epochs))

  total_train_loss = 0

  model.train()

  for step, batch in enumerate(train_loader):
    if step % 10 == 0 and not step == 0:
      print("Batch {} in {}".format(step, len(train_loader)))

    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_attention_mask, b_labels = batch

    optimizer.zero_grad()

    result = model(b_input_ids,
                   token_type_ids=None,
                   attention_mask=b_attention_mask,
                   labels=b_labels)

    loss, logits = result.loss, result.logits

    total_train_loss += loss.item()

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()

  avg_train_loss = total_train_loss / len(train_loader)
  print("Avg train loss: {}".format(avg_train_loss))

  model.eval()

  total_eval_acc = 0
  total_eval_loss = 0
  nb_eval_steps = 0

  for batch in val_loader:

    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_attention_masks, b_labels = batch

    with torch.no_grad():

      result = model(b_input_ids,
                     token_type_ids=None,
                     attention_mask=b_attention_masks,
                     labels=b_labels)

      loss, logits = result.loss, result.logits

      total_eval_loss += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      total_eval_acc += flat_acc(logits, label_ids)

  avg_val_acc = total_eval_acc/len(val_loader)
  avg_val_loss = total_eval_loss/len(val_loader)

  print("Avg val acc: {}".format(avg_val_acc))
  print("Avg val loss: {}".format(avg_val_loss))


Epoch 1 in 3 total
Batch 10 in 43
Batch 20 in 43
Batch 30 in 43
Batch 40 in 43
Avg train loss: 0.40685592834339585
Avg val acc: 0.910054347826087
Avg val loss: 0.27939615249633787
Epoch 2 in 3 total
Batch 10 in 43
Batch 20 in 43
Batch 30 in 43
Batch 40 in 43
Avg train loss: 0.21028378848419632
Avg val acc: 0.9225543478260869
Avg val loss: 0.2649670571088791
Epoch 3 in 3 total
Batch 10 in 43
Batch 20 in 43
Batch 30 in 43
Batch 40 in 43
Avg train loss: 0.15706715351620384
Avg val acc: 0.9225543478260869
Avg val loss: 0.2723681926727295


In [ ]:
df_eval = pd.read_csv('./eval.csv')
df_eval.sample(10)

,item_id,shop_id,comment,rate_star,label
162,18869902851,539031967,Giao hàng đến nhanh nghe oke nhưng_mà giao buổ...,5,0
42,20269902330,539031967,Chất_lượng sản_phẩm : không tạm ổn .,2,1
90,20867308039,539031967,Chất_lượng âm_thanh thì so với giá tiền là Ok ...,5,0
60,22409527877,539031967,Ổn nha.đóng gói tốt . tai nghe nghe ổn . dễ sử...,5,0
114,21069900865,539031967,"Tính_năng nổi_bật : sạc nhanh , cầm củ khá chắ...",5,0
137,18869902851,539031967,Rất ok nhé,5,0
41,23903614372,539031967,Shop gửi sai vỏ ốp tai nghe .,1,1
15,22414554277,539031967,Chất_lượng sản_phẩm : k nhu mo ta .,1,1
113,21069900865,539031967,Tính_năng nổi_bật : sạc nhanh đấy .,5,0
108,23109543948,539031967,Tính_năng nổi_bật : uy_tín có đổi trả bảo_hành .,5,0


In [ ]:
sentences = df_eval['comment'].values
labels = df_eval['label'].values

In [ ]:
print(len(input_ids[0]))

100


In [ ]:
input_ids = []
attention_masks = []

for sent in sentences:
  encoded_dict = tokenizer.encode_plus(
      sent,
      max_length=100,
      padding='max_length',
      return_attention_mask=True,
      truncation=True
  )

  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
labels = torch.tensor(labels)

In [ ]:
eval_dataset = TensorDataset(input_ids, attention_masks, labels)

eval_loader = DataLoader(eval_dataset, shuffle=True, batch_size=32)

In [ ]:
model.eval()
predictions, true_labels = [], []

for batch in eval_loader:

    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_attention_masks, b_labels = batch

    with torch.no_grad():

      result = model(b_input_ids,
                     token_type_ids=None,
                     attention_mask=b_attention_masks,
                     labels=b_labels)

      loss, logits = result.loss, result.logits

      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      predictions.append(logits)
      true_labels.append(label_ids)

In [ ]:
pred_labels = []
for pred in predictions:
  pred_labels.append(np.argmax(pred, axis=1))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(true_labels), np.concatenate(pred_labels)))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       130
           1       0.85      0.77      0.80        43

    accuracy                           0.91       173
   macro avg       0.89      0.86      0.87       173
weighted avg       0.91      0.91      0.91       173



In [ ]:
output_dir = './model_save'

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/bpe.codes',
 './model_save/added_tokens.json')

In [ ]:
!ls -l --block-size=M ./model_save/

total 518M
-rw-r--r-- 1 root root   1M Oct 13 12:37 added_tokens.json
-rw-r--r-- 1 root root   2M Oct 13 12:37 bpe.codes
-rw-r--r-- 1 root root   1M Oct 13 12:37 config.json
-rw-r--r-- 1 root root 516M Oct 13 12:37 pytorch_model.bin
-rw-r--r-- 1 root root   1M Oct 13 12:37 special_tokens_map.json
-rw-r--r-- 1 root root   1M Oct 13 12:37 tokenizer_config.json
-rw-r--r-- 1 root root   1M Oct 13 12:37 vocab.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r ./model_save/ "./drive/MyDrive/DS105"